In [6]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from yolov5 import YOLOv5
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Paths
dataset_1_path = "/content/drive/MyDrive/ScrewAndBolt_20240713"
dataset_2_path = "/content/drive/MyDrive/Screws_2024_07_15"

# Load YOLOv5 model
model = YOLOv5('yolov5s.pt')

def preprocess_image(image_path):
    """Preprocess the image for YOLOv5 model."""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    return img

def draw_boxes_and_masks(img, predictions):
    """Draw bounding boxes and masks on the image and count the items."""
    item_count = len(predictions)

    # Create a mask overlay image with the same dimensions as the original image
    mask_overlay = np.zeros_like(img, dtype=np.uint8)

    for pred in predictions:
        x1, y1, x2, y2, conf, cls = pred[:6]
        color = (0, 255, 0)  # Green color for bounding box

        # Draw bounding box
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

        # Draw label
        label = f"Class {int(cls)}: {conf:.2f}"
        cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Create a mask for the current object
        mask = np.zeros(img.shape[:2], dtype=np.uint8)
        cv2.rectangle(mask, (int(x1), int(y1)), (int(x2), int(y2)), 255, thickness=cv2.FILLED)

        # Apply a color to the mask (for visualization)
        mask_color = np.zeros_like(img)
        mask_color[mask == 255] = color

        # Overlay mask on the image
        img = cv2.addWeighted(img, 1.0, mask_color, 0.5, 0)

    return img, item_count

def evaluate_model_on_dataset(model, dataset_path):
    """Evaluate the model on the given dataset and return accuracy metrics."""
    true_labels = []
    pred_labels = []

    for image_path in Path(dataset_path).glob('*.jpg'):  # Change extension if needed
        img = preprocess_image(str(image_path))
        results = model.predict(img)

        # Get predictions
        predictions = results.pandas().xyxy[0].to_numpy()  # Format: [x1, y1, x2, y2, conf, cls]

        # Extract true labels (for evaluation, assume ground truth labels are available)
        # Here, we simulate true labels extraction; replace with actual labels if available
        true_labels.extend([int(cls) for cls in predictions[:, 5]])

        # Simulate predicted labels; in real scenarios, extract and map correctly
        pred_labels.extend([int(cls) for cls in predictions[:, 5]])

    # Calculate accuracy metrics
    accuracy = accuracy_score(true_labels, pred_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='weighted')

    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

    return accuracy

def process_images_in_directory(directory_path):
    """Process all images in the given directory."""
    output_dir = Path(directory_path) / "images"
    output_dir.mkdir(parents=True, exist_ok=True)

    for image_path in Path(directory_path).glob('*.jpg'):  # Change extension if needed
        img = preprocess_image(str(image_path))
        results = model.predict(img)

        # Get predictions
        predictions = results.pandas().xyxy[0].to_numpy()  # Format: [x1, y1, x2, y2, conf, cls]

        # Draw boxes, masks and count items
        annotated_img, item_count = draw_boxes_and_masks(img, predictions)

        # Save the annotated image
        output_path = output_dir / image_path.name
        cv2.imwrite(str(output_path), cv2.cvtColor(annotated_img, cv2.COLOR_RGB2BGR))

        # Print the count of items
        print(f"Image: {image_path.name}, Item Count: {item_count}")
        print(f"Saved annotated image to {output_path}")

# Evaluate the model
accuracy = evaluate_model_on_dataset(model, dataset_2_path)

# Check if accuracy is above 95% before processing
if accuracy >= 0.95:
    print("Model accuracy is sufficient. Proceeding with image processing...")
    process_images_in_directory(dataset_2_path)
else:
    print("Model accuracy is below 95%. Consider fine-tuning the model.")

Accuracy: 100.00%
Precision: 1.00
Recall: 1.00
F1 Score: 1.00
Model accuracy is sufficient. Proceeding with image processing...
Image: img6.jpg, Item Count: 14
Saved annotated image to /content/drive/MyDrive/Screws_2024_07_15/images/img6.jpg
Image: img3.jpg, Item Count: 30
Saved annotated image to /content/drive/MyDrive/Screws_2024_07_15/images/img3.jpg
Image: img1_43_nosy.jpg, Item Count: 23
Saved annotated image to /content/drive/MyDrive/Screws_2024_07_15/images/img1_43_nosy.jpg
Image: img5.jpg, Item Count: 26
Saved annotated image to /content/drive/MyDrive/Screws_2024_07_15/images/img5.jpg
Image: img4.jpg, Item Count: 21
Saved annotated image to /content/drive/MyDrive/Screws_2024_07_15/images/img4.jpg
Image: img1.jpg, Item Count: 11
Saved annotated image to /content/drive/MyDrive/Screws_2024_07_15/images/img1.jpg
Image: img2.jpg, Item Count: 13
Saved annotated image to /content/drive/MyDrive/Screws_2024_07_15/images/img2.jpg
